# ML/AI Engineering Foundations: Final Portfolio
The following Jupyter notebook was created as a part of the final project for\
the Codecademy course 
["ML/AI Engineering Foundations"](https://www.codecademy.com/learn/paths/machine-learning|ai|engineering|foundations). 

## Topic
This project will investigate the relationship between book genre and sales \
over time. In order to do this the following dataset will be used:\
[Best-selling books dataset](https://www.kaggle.com/datasets/mansivasantpatel/best-selling-books-dataset/data)

This dataset is described by it's author as the following:
> This dataset provides lists of best-selling books and book series to date and\
> in any language. "Best-selling" refers to the estimated number of copies sold\
> of each book, rather than the number of books printed. Comics and textbooks\
> are not included in this list.

This data set contains the following fields:
| Field             | Description                                           |
| ----------------- | ----------------------------------------------------- |
|Index              | row index to identify each row uniquely.              |
|Book               | Contains title of the book(s).                        |
|Author             | Contains Name of the author(s) who wrote the book.    |
|Original language  | Language in which the book was originally written.    |
|First published    | The year in which the book was first published.       |
|Approximate sales  | Approximate sales of the book.                        |
|Genre              | Genre of the book.                                    |

The bellow cell is used in to import the dataset (not included within the\
repository as including the dataset this way can be bad practice, as a dataset\
can easily be very large and complex):

In [2]:
import kaggle

kaggle.api.authenticate()

kaggle.api.dataset_download_files(dataset = 'mansivasantpatel/best-selling-books-dataset', path = './', unzip = True)

## Data Analysis
With the dataset selected, the next step of the project is to determine how this dataset might be analysed. 